In [45]:
# Prepare training ingredients-cuisines dataframe

train_dataframe_file_path = os.path.join(data_directory, "train.pkl")

if(not pathlib.Path(train_dataframe_file_path).is_file()):
    
    # Warning!!! Avoid runing this if existing dataframe exists, check
    # for .pkl file in Dataset folder
    
    columnLabels = np.array(['ingredients', 'cuisines'])

    trainDf = pd.DataFrame(columns=columnLabels)

    print("index type: " + str(trainDf.index.dtype))

    # Populate and save ingredients-cuisines dataframe

    index = 0;
    for dish in trainData:
        ingredientStr = " ".join(dish['ingredients'])
        trainDf.loc[index] = [ingredientStr, dish['cuisine']]
        index+=1
        
    trainDf.to_pickle(train_dataframe_file_path)
    
else:
    
    trainDf = pd.read_pickle(train_dataframe_file_path)

trainDf.head()

,ingredients,cuisines
0,romaine lettuce black olives grape tomatoes ga...,greek
1,plain flour ground pepper salt tomatoes ground...,southern_us
2,eggs pepper salt mayonaise cooking oil green c...,filipino
3,water vegetable oil wheat salt,indian
4,black pepper shallots cornflour cayenne pepper...,indian


In [46]:
# Prepare testing ingredients-cuisines dataframe

test_dataframe_file_path = os.path.join(data_directory, "test.pkl")

if(not pathlib.Path(test_dataframe_file_path).is_file()):
    
    # Warning!!! Avoid runing this if existing dataframe exists, check
    # for .pkl file in Dataset folder
    
    columnLabels = np.array(['ingredients'])

    testDf = pd.DataFrame(columns=columnLabels)

    print("index type: " + str(testDf.index.dtype))

    # Populate and save ingredients-cuisines dataframe

    index = 0;
    for dish in testData:
        testDf.loc[index] = " ".join(dish['ingredients'])
        index+=1
        
    testDf.to_pickle(test_dataframe_file_path)
    
else:
    
    testDf = pd.read_pickle(test_dataframe_file_path)

testDf.head()

,ingredients
0,baking powder eggs all-purpose flour raisins m...
1,sugar egg yolks corn starch cream of tartar ba...
2,sausage links fennel bulb fronds olive oil cub...
3,meat cuts file powder smoked sausage okra shri...
4,ground black pepper salt sausage casings leeks...


In [14]:
# (Feel free to skip this part)
# Extract unique ingredients and cuisines words

ingredients = set()
cuisines = set()

for dish in trainData:
    cuisines.add(dish['cuisine'])
    for ingred in dish['ingredients']:
        ingredients.add(ingred)

prettyPrinter.pprint('no. unique ingredients: '+ str(len(ingredients)))
prettyPrinter.pprint('no. unique cuisines: '+ str(len(cuisines)))

unique_ingredients_data_file_path = os.path.join(data_directory, "unique_ingredients.txt")
unique_cuisines_data_file_path = os.path.join(data_directory, "unique_cuisines.txt")

# Writing unique ingredients txt file
with open(unique_ingredients_data_file_path, 'w') as file:
     file.write('\n'.join(ingredients))
file.closed

# Writing unique cuisines txt file
with open(unique_cuisines_data_file_path, 'w') as file:
     file.write('\n'.join(cuisines))
file.closed

'no. unique ingredients: 6714'
'no. unique cuisines: 20'


True

In [34]:
train = {"data": trainDf.ingredients, 
         "target": trainDf.cuisines}

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train['data'])
print(train_counts.shape)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_counts)
print(train_tfidf.shape)

(39774, 3010)
(39774, 3010)


In [35]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', OneVsRestClassifier(MultinomialNB()))])
text_clf = text_clf.fit(train['data'], train['target'])

In [36]:
import numpy as np
test = {"data": toxic_test_raw.comment_text, 
         "target": train['target']}

predicted = text_clf.predict(test['data'])
np.mean(predicted[0:20] == test['target'][0:20])

NameError: name 'toxic_test_raw' is not defined

In [8]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', 
                                                   penalty='l2',
                                                   alpha=1e-3,
                                                   n_iter=5,
                                                   random_state=42)),
                         ])
_ = text_clf_svm.fit(train['data'], train['target'])
predicted_svm = text_clf_svm.predict(test['data'])
np.mean(predicted_svm == test['target'])

/home/andrew/Documents/COMP6208/AML_CW/env/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.91538562771430898